In [2]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import cv2
import pandas as pd
import pdb
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.mylayer import mylayer
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras_frcnn.RoiPoolingConv import RoiPoolingConv
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout

gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

K.set_learning_phase(1) #set learning phase

from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

print 'loading dataframe'
df = pd.read_pickle("./upper_body_atts.pkl")
#df= df[0:10000]
df = df[['img_name','landmarks','atts']]

df["labels"] = df["atts"].apply(lambda x:list(x))

def name(x):
    name = str.split(x,'/')
    return '/Users/1024329/Downloads/DeepFashion/img_n/'+ name[1]+'/'  + name[2]
    #return '/fashion-attribute/storage_bucket/deep_fashion/img_bbox_data/img_n/' + name[1] + '/' + name[2]
#/fashion-attribute/storage_bucket/small_data_set/work/train/Blazer/Double-Breasted_Blazer

# local path : /Users/1024329/Downloads/DeepFashion/img_n/
# server path : /fashion-attribute/storage_bucket/deep_fashion/img_bbox_data/img_n/
    
df['Filenames'] = df['img_name'].apply(lambda x:name(x))

#df['Filenames'] = df['img_name'].apply(lambda x: '/Users/1024329/Downloads/DeepFashion/' + x)

# removing the samples without any attribute present in it
sample = df['atts'].apply(lambda x: x.shape[0]>0)
df4 =df[sample]


/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


loading dataframe


In [3]:
df_cat = pd.read_csv('/Users/1024329/Downloads/DeepFashion/processed_data/list_attr_cloth.csv')

In [4]:
select = np.array(df_cat[df_cat.att_type == 1 ].index)
select.shape

(156,)

In [5]:
df4['cat1atts'] = df4['labels'].apply(lambda x: [item for item in select if item in x])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
sample = df4['cat1atts'].apply(lambda x: len(x)>0)
df5 =df4[sample]

In [11]:
df5 = df5.reset_index(drop= True)

In [18]:
vals = np.array([])
for i in range(df5.shape[0]):
    if i%10000 ==0: print i
    vals = np.concatenate([vals,df5.loc[i,'cat1atts']]) 

0
10000
20000
30000
40000
50000


In [ ]:
y = np.bincount(vals)
ii = np.nonzero(y)[0]

In [40]:
s = zip(ii,y[ii])

In [60]:
sa = [ i[0] for i in s if i[1] > 1000]

In [68]:
print len(sa)

15


In [63]:
df5['cat1atts'] = df5['cat1atts'].apply(lambda x: [item for item in sa if item in x])

In [65]:
sample = df5['cat1atts'].apply(lambda x: len(x)>0)
df5 =df5[sample]

In [66]:
print df5.shape

(44163, 7)


In [67]:

print 'creating image generators'
#import pdb
#pdb.set_trace()
img_reso = 224
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(dataframe=df5[0:37000],directory="", 
        x_col="Filenames",y_col="cat1atts",batch_size=32,
        seed=42,shuffle=True,class_mode="categorical",
        classes=list(sa),target_size=(img_reso,img_reso))

print 'validation generator'
valid_generator=test_datagen.flow_from_dataframe(
        dataframe=df5[37000:40000],
directory="",
x_col="Filenames",
y_col="cat1atts",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=list(sa),
target_size=(img_reso,img_reso))

print 'test generator'
test_generator=test_datagen.flow_from_dataframe(dataframe=df5[40000:],
directory="",
x_col="Filenames",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(img_reso,img_reso))


creating image generators
Found 37000 validated image filenames belonging to 15 classes.
validation generator
Found 3000 validated image filenames belonging to 15 classes.
test generator
Found 4163 validated image filenames.


In [69]:

network ='vgg'

if network == 'vgg':
	from keras_frcnn import vgg as nn
elif network == 'resnet50':
	from keras_frcnn import resnet as nn
else:
	print('Not a valid model')
	raise ValueError


if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (224,224, 3)

img_input = Input(shape=input_shape_img)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=False)

out = (Flatten(name='flatten'))(shared_layers)
out = (Dense(4096, activation='relu', name='fc5'))(out)
out = (Dropout(0.5))(out)
out = (Dense(2048, activation='relu', name='fc6'))(out)
out = (Dropout(0.5))(out)
out = (Dense(512, activation='relu', name='fc7'))(out)
out_class = (Dense(15, activation='sigmoid'))(out)

model_classifier = Model(img_input, out_class)

try:
	print('loading weights from pretrained model')
	model_classifier.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels.h5', by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')

model_classifier.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])


checkpoint = ModelCheckpoint("cat1_atts.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model_classifier.fit_generator(generator=train_generator,
                   steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=valid_generator,
                   validation_steps=STEP_SIZE_VALID,
                   epochs=50, use_multiprocessing=True, callbacks = [checkpoint])




Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
loading weights from pretrained model
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
   1/1156 [..............................] - ETA: 10:16:04 - loss: 0.8182 - acc: 0.5271

Process PoolWorker-1:
Process PoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
    task = get()
  File "/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
    return recv()
  File "/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in get
    return recv()
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: 

In [4]:
model_classifier.load_weights('vgg16_atts.h5')

In [9]:
# import keras
# model_f = keras.models.load_model('vgg16_atts.h5')

In [6]:
a, b = train_generator.next()

In [7]:
a.shape, b.shape

((32, 224, 224, 3), (32, 1000))

In [8]:
out = model_classifier.predict(a)

In [9]:
out.shape

(32, 1000)

In [27]:
num = 10
np.nonzero(out[num,:]> 0.1), np.nonzero(b[num,:]> 0.4)

((array([365, 730, 822, 836]),), (array([226, 437, 544, 545, 730]),))